# Домашнее задание по теме «Рекомендации на основе содержания»

Владимир Никифоров

Использовать dataset MovieLens https://grouplens.org/datasets/movielens/latest/
-  Построить рекомендации (регрессия, предсказываем оценку) для пользователя на фичах:
-  отдельно TF-IDF на тегах и жанрах
-  Средние оценки (+ median, variance, etc.) пользователя и фильма
-  Оценить RMSE на тестовой выборке

## Загружаем данные

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, Ridge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

from tqdm import tqdm_notebook

RANDOM_STATE = 7

In [2]:
movies = pd.read_csv('../data/ml-latest-small/movies.csv')
tags = pd.read_csv('../data/ml-latest-small/tags.csv')
ratings = pd.read_csv('../data/ml-latest-small/ratings.csv')

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [5]:
movies['genres_for_tfidf'] = movies.apply(lambda r: ' '.join(r['genres'].split('|')), axis=1)

In [6]:
movies['tags_for_tfidf'] = movies.apply(lambda r: ' '.join(tags[tags['movieId']==r['movieId']]['tag'].values), axis=1)

In [7]:
movies.drop('genres',axis=1,inplace=True)
movies.head()

,movieId,title,genres_for_tfidf,tags_for_tfidf
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,pixar pixar fun
1,2,Jumanji (1995),Adventure Children Fantasy,fantasy magic board game Robin Williams game
2,3,Grumpier Old Men (1995),Comedy Romance,moldy old
3,4,Waiting to Exhale (1995),Comedy Drama Romance,
4,5,Father of the Bride Part II (1995),Comedy,pregnancy remake


In [8]:
def get_tfidf(df, fld):
    cnt_vec = CountVectorizer()
    processed = cnt_vec.fit_transform(df[fld])
    tfidf = TfidfTransformer()
    tfidf_dense = tfidf.fit_transform(processed).todense()
    columns = [None for i in range(len(cnt_vec.vocabulary_))]
    for k in cnt_vec.vocabulary_:
        columns[cnt_vec.vocabulary_[k]] = k
    df_tfidf = pd.DataFrame(tfidf_dense, columns=columns)
    return pd.concat((df.drop(fld,1), df_tfidf), axis=1)

Применим tf-idf и на тэгах, и на жанрах

In [9]:
movies_with_tfidf = get_tfidf(movies, 'genres_for_tfidf')
movies_with_tfidf = get_tfidf(movies_with_tfidf, 'tags_for_tfidf')
movies_with_tfidf.head()

,movieId,title,action,adventure,animation,children,comedy,crime,documentary,drama,...,york,you,younger,your,zellweger,zither,zoe,zombie,zombies,zooey
0,1,Toy Story (1995),0.0,0.416846,0.516225,0.504845,0.267586,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),0.0,0.512361,0.000000,0.620525,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),0.0,0.000000,0.000000,0.000000,0.570915,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),0.0,0.000000,0.000000,0.000000,0.505015,0.0,0.0,0.466405,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
movies_with_tfidf.columns

Index(['movieId', 'title', 'action', 'adventure', 'animation', 'children',
       'comedy', 'crime', 'documentary', 'drama',
       ...
       'york', 'you', 'younger', 'your', 'zellweger', 'zither', 'zoe',
       'zombie', 'zombies', 'zooey'],
      dtype='object', length=1770)

In [11]:
fc = [c for c in movies_with_tfidf.columns if c not in ['genres', 'genres_for_tfidf', 'tags_for_tfidf']]

In [12]:
movies_for_ds = movies_with_tfidf[fc]

In [13]:
movies_for_ds

,movieId,title,action,action,adventure,adventure,animation,animation,children,children,...,york,you,younger,your,zellweger,zither,zoe,zombie,zombies,zooey
0,1,Toy Story (1995),0.000000,0.0,0.416846,0.0,0.516225,0.0,0.504845,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),0.000000,0.0,0.512361,0.0,0.000000,0.0,0.620525,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6,Heat (1995),0.549328,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,7,Sabrina (1995),0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,8,Tom and Huck (1995),0.000000,0.0,0.636699,0.0,0.000000,0.0,0.771112,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,9,Sudden Death (1995),1.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,10,GoldenEye (1995),0.553065,0.0,0.629522,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## CBRS For User

In [14]:
ratings = pd.read_csv('../data/ml-latest-small/ratings.csv')

In [15]:
movies_with_ratings = pd.merge(ratings, movies_for_ds, on='movieId')

In [16]:
movies_with_ratings.head()

,userId,movieId,rating,timestamp,title,action,action,adventure,adventure,animation,...,york,you,younger,your,zellweger,zither,zoe,zombie,zombies,zooey
0,1,1,4.0,964982703,Toy Story (1995),0.0,0.0,0.416846,0.0,0.516225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5,1,4.0,847434962,Toy Story (1995),0.0,0.0,0.416846,0.0,0.516225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,7,1,4.5,1106635946,Toy Story (1995),0.0,0.0,0.416846,0.0,0.516225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15,1,2.5,1510577970,Toy Story (1995),0.0,0.0,0.416846,0.0,0.516225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,17,1,4.5,1305696483,Toy Story (1995),0.0,0.0,0.416846,0.0,0.516225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Для прогнозирования рейтинга для конкретного пользователя обучимся только на его фильмах

In [17]:
ratings.groupby('userId')[['movieId']].count().sort_values('movieId', ascending=False).head()

,movieId
userId,
414,2698
599,2478
474,2108
448,1864
274,1346


Выбираем пользователя с большим количеством оцененных фильмов

In [18]:
TARGET_USER = 414

In [19]:
def get_prediction(df, target_user, random_state = RANDOM_STATE):
    # Процедура для тренировки на фильмах, оцененных заданным пользователем, с валидацией
    # и прогнозом на отложенной выборке строк для сравнения с реальными оценками пользователя
    df = df[df['userId']==target_user]
    drop_col_list = ['userId','movieId','timestamp','title','rating']
    df_for_user, df_for_user_test = train_test_split(df, test_size = 0.1, random_state = random_state)
    X, y = df_for_user.drop(drop_col_list,axis=1,inplace=False), df_for_user['rating']
    del df_for_user
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state = random_state)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_valid = sc.transform(X_valid)
    X_test = sc.transform(df_for_user_test.drop(drop_col_list,axis=1,inplace=False))
    
    models = [Lasso, Ridge, DecisionTreeRegressor, SVR, RandomForestRegressor]
    models_results = {}
    for m in tqdm_notebook(models):
        model = m()
        model.fit(X_train, y_train)
        models_results[m.__name__] = [model,
                                      model.score(X_train ,y_train),
                                      model.score(X_valid, y_valid),
                                      mean_squared_error(model.predict(X_train), y_train)**.5,
                                      mean_squared_error(model.predict(X_valid), y_valid)**.5
                                     ]
        print("{}. r2_train: {:.4f}, r2_test: {:.4f}, rmse_train: {:.4f}, rmse_test: {:.4f}".format(
            m.__name__, 
            models_results[m.__name__][1],
            models_results[m.__name__][2],
            models_results[m.__name__][3],
            models_results[m.__name__][4]
        ))
    # find best model and return it prediction to X_test
    best_ = sorted(models_results.values(), key=lambda t: t[4], reverse=False)[0]
    best_model = best_[0]
    print('-'*100)
    print(f'BEST VALIDATION_RMSE = {best_[4]}\nMODEL = {best_model}')
    rslt = pd.concat([df_for_user_test.reset_index().drop('index',1),
                      pd.DataFrame(columns=['predicted_rating'],data=best_model.predict(X_test))],
                     axis=1)
    print(f"RMSE TEST = {mean_squared_error(rslt['predicted_rating'], rslt['rating'])**.5}")
    return rslt[drop_col_list + ['predicted_rating'] + [c for c in rslt.columns if c not in drop_col_list + ['predicted_rating']]]

In [20]:
%%time
target_user_ratings = get_prediction(movies_with_ratings, TARGET_USER)

Lasso. r2_train: 0.0000, r2_test: -0.0005, rmse_train: 0.9309, rmse_test: 0.9309
Ridge. r2_train: 0.3431, r2_test: -0.0142, rmse_train: 0.7545, rmse_test: 0.9373
DecisionTreeRegressor. r2_train: 0.5294, r2_test: -0.1449, rmse_train: 0.6386, rmse_test: 0.9958


/home/truename/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVR. r2_train: 0.3074, r2_test: 0.1117, rmse_train: 0.7747, rmse_test: 0.8772


/home/truename/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor. r2_train: 0.4551, r2_test: -0.0315, rmse_train: 0.6872, rmse_test: 0.9452

----------------------------------------------------------------------------------------------------
BEST VALIDATION_RMSE = 0.8771531140438756
MODEL = SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)
RMSE TEST = 0.8993235806291743
CPU times: user 35 s, sys: 2.29 s, total: 37.3 s
Wall time: 32 s


In [21]:
target_user_ratings.head()

,userId,movieId,timestamp,title,rating,predicted_rating,action,action,action,action,...,york,you,younger,your,zellweger,zither,zoe,zombie,zombies,zooey
0,414,205,961438413,Unstrung Heroes (1995),3.0,3.399547,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,414,1257,961595667,Better Off Dead... (1985),5.0,2.928863,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,414,6,961515642,Heat (1995),3.0,3.033029,0.549328,0.0,0.549328,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,414,2724,1025108900,Runaway Bride (1999),1.0,2.928232,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,414,43928,1216151152,Ultraviolet (2006),2.5,3.025534,0.384787,0.0,0.384787,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Добавим средние оценки пользователя и фильмов для обучения по всем пользователям

In [22]:
users_rating = ratings.groupby('userId')['rating'].agg(['mean','median','std','var']).reset_index()
movie_rating = ratings.groupby('movieId')['rating'].agg(['mean','median','std','var']).reset_index()

In [23]:
movies_with_ratings.head()

,userId,movieId,rating,timestamp,title,action,action,adventure,adventure,animation,...,york,you,younger,your,zellweger,zither,zoe,zombie,zombies,zooey
0,1,1,4.0,964982703,Toy Story (1995),0.0,0.0,0.416846,0.0,0.516225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5,1,4.0,847434962,Toy Story (1995),0.0,0.0,0.416846,0.0,0.516225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,7,1,4.5,1106635946,Toy Story (1995),0.0,0.0,0.416846,0.0,0.516225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15,1,2.5,1510577970,Toy Story (1995),0.0,0.0,0.416846,0.0,0.516225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,17,1,4.5,1305696483,Toy Story (1995),0.0,0.0,0.416846,0.0,0.516225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
%%time
# показатели для пользователя при фильтрации датасета по конкретному пользователю, тренировке и предсказанию
# по конкретному пользователю смысла не имеют
movies_with_ratings = movies_with_ratings.merge(users_rating, on='userId')
movies_with_ratings = movies_with_ratings.merge(movie_rating, on='movieId')
del users_rating, movie_rating

CPU times: user 5.33 s, sys: 4.52 s, total: 9.85 s
Wall time: 8.36 s


In [25]:
movies_with_ratings.head()

,userId,movieId,rating,timestamp,title,action,action,adventure,adventure,animation,...,zombies,zooey,mean_x,median_x,std_x,var_x,mean_y,median_y,std_y,var_y
0,1,1,4.0,964982703,Toy Story (1995),0.0,0.0,0.416846,0.0,0.516225,...,0.0,0.0,4.366379,5.0,0.800048,0.640077,3.92093,4.0,0.834859,0.69699
1,5,1,4.0,847434962,Toy Story (1995),0.0,0.0,0.416846,0.0,0.516225,...,0.0,0.0,3.636364,4.0,0.990441,0.980973,3.92093,4.0,0.834859,0.69699
2,7,1,4.5,1106635946,Toy Story (1995),0.0,0.0,0.416846,0.0,0.516225,...,0.0,0.0,3.230263,3.5,1.329594,1.767820,3.92093,4.0,0.834859,0.69699
3,15,1,2.5,1510577970,Toy Story (1995),0.0,0.0,0.416846,0.0,0.516225,...,0.0,0.0,3.448148,3.5,1.133404,1.284605,3.92093,4.0,0.834859,0.69699
4,17,1,4.5,1305696483,Toy Story (1995),0.0,0.0,0.416846,0.0,0.516225,...,0.0,0.0,4.209524,4.0,0.508490,0.258562,3.92093,4.0,0.834859,0.69699


In [26]:
def get_prediction2(df, target_user, random_state = RANDOM_STATE):
    # Процедура для тренировки на фильмах, оцененных всеми пользователеми, с валидацией
    # и прогнозом на отложенной выборке строк для сравнения с реальными оценками пользователя
    df = df[df['userId']==target_user]
    drop_col_list = ['userId','movieId','timestamp','title','rating']
    df_for_user, df_for_user_test = train_test_split(df, test_size = 0.1, random_state = random_state)
    X, y = df_for_user.drop(drop_col_list,axis=1,inplace=False), df_for_user['rating']
    del df_for_user
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state = random_state)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_valid = sc.transform(X_valid)
    X_test = sc.transform(df_for_user_test.drop(drop_col_list,axis=1,inplace=False))
    
    models = [Lasso, Ridge, DecisionTreeRegressor, SVR, RandomForestRegressor]
    models_results = {}
    for m in tqdm_notebook(models):
        model = m()
        model.fit(X_train, y_train)
        models_results[m.__name__] = [model,
                                      model.score(X_train ,y_train),
                                      model.score(X_valid, y_valid),
                                      mean_squared_error(model.predict(X_train), y_train)**.5,
                                      mean_squared_error(model.predict(X_valid), y_valid)**.5
                                     ]
        print("{}. r2_train: {:.4f}, r2_test: {:.4f}, rmse_train: {:.4f}, rmse_test: {:.4f}".format(
            m.__name__, 
            models_results[m.__name__][1],
            models_results[m.__name__][2],
            models_results[m.__name__][3],
            models_results[m.__name__][4]
        ))
    # find best model and return it prediction to X_test
    best_ = sorted(models_results.values(), key=lambda t: t[4], reverse=False)[0]
    best_model = best_[0]
    print('-'*100)
    print(f'BEST VALIDATION_RMSE = {best_[4]}\nMODEL = {best_model}')
    rslt = pd.concat([df_for_user_test.reset_index().drop('index',1),
                      pd.DataFrame(columns=['predicted_rating'],data=best_model.predict(X_test))],
                     axis=1)
    print(f"RMSE TEST = {mean_squared_error(rslt['predicted_rating'], rslt['rating'])**.5}")
    return rslt[drop_col_list + ['predicted_rating'] + [c for c in rslt.columns if c not in drop_col_list + ['predicted_rating']]]

In [27]:
movies_with_ratings.fillna(0,inplace=True)

In [30]:
%%time
predicted_ratings = get_prediction2(movies_with_ratings, TARGET_USER)

Lasso. r2_train: 0.0000, r2_test: -0.0094, rmse_train: 0.9397, rmse_test: 0.9315
Ridge. r2_train: 0.6309, r2_test: 0.4208, rmse_train: 0.5709, rmse_test: 0.7056
DecisionTreeRegressor. r2_train: 0.9907, r2_test: 0.0237, rmse_train: 0.0904, rmse_test: 0.9161


/home/truename/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVR. r2_train: 0.5978, r2_test: 0.4602, rmse_train: 0.5959, rmse_test: 0.6812


/home/truename/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor. r2_train: 0.8959, r2_test: 0.4401, rmse_train: 0.3032, rmse_test: 0.6937

----------------------------------------------------------------------------------------------------
BEST VALIDATION_RMSE = 0.6811786847374233
MODEL = SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)
RMSE TEST = 0.6767400678131588
CPU times: user 30.8 s, sys: 457 ms, total: 31.3 s
Wall time: 27.9 s


RMSE на ТЕСТе заметно улучшился при добавлении метрик по фильмам

In [31]:
predicted_ratings.head()

,userId,movieId,timestamp,title,rating,predicted_rating,action,action,action,action,...,zombies,zooey,mean_x,median_x,std_x,var_x,mean_y,median_y,std_y,var_y
0,414,1093,961436657,"Doors, The (1991)",3.0,3.935945,0.000000,0.0,0.000000,0.0,...,0.0,0.0,3.391957,3.5,0.933307,0.871062,3.405405,3.5,0.991897,0.983859
1,414,59369,1283877039,Taken (2008),3.5,3.728646,0.515078,0.0,0.515078,0.0,...,0.0,0.0,3.391957,3.5,0.933307,0.871062,3.619048,4.0,1.103263,1.217189
2,414,6,961515642,Heat (1995),3.0,3.819551,0.549328,0.0,0.549328,0.0,...,0.0,0.0,3.391957,3.5,0.933307,0.871062,3.946078,4.0,0.817224,0.667856
3,414,3827,989248906,Space Cowboys (2000),2.0,2.996597,0.412730,0.0,0.412730,0.0,...,0.0,0.0,3.391957,3.5,0.933307,0.871062,2.891304,3.0,0.811235,0.658103
4,414,41285,1216147618,Match Point (2005),3.0,3.475690,0.000000,0.0,0.000000,0.0,...,0.0,0.0,3.391957,3.5,0.933307,0.871062,3.441176,3.5,0.966345,0.933824
